#get data

In [1]:
!gdown 1Y6A1SHDnoNA3L_kRZOy71wz7duApMhPx

Downloading...
From: https://drive.google.com/uc?id=1Y6A1SHDnoNA3L_kRZOy71wz7duApMhPx
To: /content/sum_sh.c
100% 1.27k/1.27k [00:00<00:00, 3.39MB/s]


In [2]:
PATH = "/content/sum_sh.c"

#utils

In [ ]:
import re

In [ ]:
def detect_functions(code):
    code = preprocess_code(code)



    # Match lines that may contain function declarations
    potential_function_lines = [
        line for line in code.split('\n') if re.match(r'\s*[a-zA-Z_]\w*\s+[a-zA-Z_]\w*\s*\([^)]*\)\s*{?', line)
    ]

    # Filter out lines starting with "if", "else if", or "else"
    potential_function_lines = [
        line for line in potential_function_lines if not re.match(r'\s*(if|else if|else)\s*\(', line)
    ]

    function_names = []
    for line in potential_function_lines:
        match = re.match(r'\s*[a-zA-Z_]\w*\s+([a-zA-Z_]\w*)\s*\([^)]*\)\s*{?', line)
        if match:
            function_names.append(match.group(1))
    return function_names

In [ ]:
def preprocess_code(code):
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
    code = re.sub(r'//.*', '', code)
    return code

In [ ]:
def detect_conditions(code):
    code = preprocess_code(code)
    # Identify lines that may contain conditions
    condition_lines = [line.strip() for line in code.split('\n') if re.match(r'\s*if\s*\(|\s*else if\s*\(|\s*else\s*{?', line)]
    condition_lines1 = [line.strip() for line in code.split('\n') if re.match(r'\s*(if|else if)\s*[^{]*\s*{?', line)]
    condition_lines1 = [re.sub(r'\([^)]*\)', '', line) for line in condition_lines]
    # Extract conditions and their nesting levels
    conditions = []
    current_indent = 0
    for line in condition_lines:
        match_if = re.match(r'\s*if\s*\((.*)\)\s*{?', line)
        match_elif = re.match(r'\s*else if\s*\((.*)\)\s*{?', line)
        match_else = re.match(r'\s*else\s*{?', line)

        if match_if:
            condition = {'type': 'if', 'condition': match_if.group(1), 'indent': current_indent}
            conditions.append(condition)
            current_indent += 1
        elif match_elif:
            condition = {'type': 'elif', 'condition': match_elif.group(1), 'indent': current_indent}
            conditions.append(condition)
        elif match_else:
            condition = {'type': 'else', 'condition': None, 'indent': current_indent}
            conditions.append(condition)
            current_indent -= 1

    return conditions

In [ ]:
def count_brackets(line):
    opening_brackets = re.findall(r'{', line)
    closing_brackets = re.findall(r'}', line)
    return len(opening_brackets), len(closing_brackets)

In [ ]:
def extract_function(function, code):
  declarations = [f"{i} {function}" for i in ['int', 'bool', 'float', 'double', 'void']]
  flag = False
  opening_count = 0
  closing_count = 0
  function_code = """"""
  for line in code:
    if not flag:
      for declaration in declarations:
        if declaration in line:
          flag=True
          o, c = count_brackets(line)
          opening_count += o
          closing_count += c

          function_code += line
          break
    else:
      o, c = count_brackets(line)
      opening_count += o
      closing_count += c

      function_code += line
    if opening_count !=0 and opening_count == closing_count:
      return {"name": function, "body": function_code}

In [ ]:
def count_function_calls(code_content, function_name):
    pattern = re.compile(r'\b' + re.escape(function_name) + r'\s*\(')
    matches = pattern.findall(code_content)
    return len(matches)-1

In [ ]:
def detect_loops(code):
    code = preprocess_code(code)
    lines = code.split('\n')

    loops = []
    current_indent = 0  # Track the current nesting level
    indent_stack = []  # Stack to manage indent levels

    for line in lines:
        line = line.strip()

        # Increase indent level when a '{' is found
        if '{' in line:
            indent_stack.append('{')
            current_indent = len(indent_stack) - 1

        # Detect 'for' and 'while' loops
        match_for = re.match(r'\s*for\s*\((.*)\)\s*{?', line)
        match_while = re.match(r'\s*while\s*\((.*)\)\s*{?', line)

        if match_for or match_while:
            loop_type = 'for' if match_for else 'while'
            condition = match_for.group(1) if match_for else match_while.group(1)
            loop = {'type': loop_type, 'condition': condition, 'indent': current_indent}
            loops.append(loop)

        # Decrease indent level when a '}' is found
        if '}' in line:
            if indent_stack and indent_stack[-1] == '{':
                indent_stack.pop()
            current_indent = len(indent_stack) - 1

    return loops

#driver

In [ ]:
function_details = []
with open(PATH) as f:
  code_lines = f.readlines()

  code = ''
  for line in code_lines:
    code += "\n"+line

  code_string = preprocess_code(code)
  functions = detect_functions(code_string)


  for function in functions:
    temp = extract_function(function, code_lines)
    temp["call_count"] = count_function_calls(code_string, function)

    temp["conditions"] = detect_conditions(temp["body"])
    temp["condition_count"] = len(temp["conditions"])

    temp["loops"] = detect_loops(temp["body"])
    temp["loop_count"] = len(temp["loops"])

    function_details.append(temp)

In [ ]:
for function in function_details:
  print(f"function name: {function['name']}")

  print(f"this function was called {function['call_count']} times")

  print(f"condition count: {function['condition_count']}")
  for condition in function['conditions']:
    print(f"\tType: {condition['type']}, Condition: {condition['condition']}")

  print(f"loop count; {function['loop_count']}")
  ind = 0
  for loop in function['loops']:
    ind = max(ind, loop['indent'])
    print(f"\tType: {loop['type']}, Condition: {loop['condition']}, Indent: {loop['indent']}")

  print(f"Time complexity: {'1' if ind == 0 else 'n^'+str(ind)}")
  print("\n\n")